# Overview


Tsim is a quantum circuit sampler that can efficiently sample from Clifford+T circuits with Pauli noise. It is based on ZX-calculus stabilizer rank decomposition and parametrized ZX diagrams, following work of [arXiv:2403.06777](https://arxiv.org/abs/2403.06777).

## Supported Gates

Tsim supports a universal gate set, together with measurement and reset instructions, and Pauli noise channels.

### Clifford Instructions

Tsim supports all instructions [supported by STIM](https://github.com/quantumlib/Stim/wiki/Stim-v1.9-Gate-Reference). Below we show the standard generating set of Clifford gates:

In [1]:
from tsim import Circuit

c = Circuit("H 0")
print(c.to_matrix())
c.diagram("pyzx", height=120);

[[ 0.70710678+0.j  0.70710678+0.j]
 [ 0.70710678+0.j -0.70710678+0.j]]


In [2]:
c = Circuit("S 0")
print(c.to_matrix())
c.diagram("pyzx", height=120)

[[ 1.+0.j  0.-0.j]
 [-0.+0.j  0.+1.j]]


Graph(3 vertices, 2 edges)

In [3]:
c = Circuit("CNOT 0 1")
print(c.to_matrix())
c.diagram("pyzx", height=160)

[[ 1.+0.j  0.+0.j -0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j -0.+0.j]
 [ 0.+0.j -0.+0.j  0.+0.j  1.+0.j]
 [-0.+0.j  0.+0.j  1.+0.j  0.+0.j]]


Graph(6 vertices, 5 edges)

### Non-Clifford Instructions

In addition to Clifford gates, Tsim supports the following non-Clifford gates. Note that all rotation parameters are defined in units of $\pi$.

Computation time and memory requirement scales exponentially with the number of non-Clifford gates.

In [4]:
c = Circuit("T 0")
print(c.to_matrix())
c.diagram("pyzx", height=120)

[[ 1.        +0.j          0.        +0.j        ]
 [-0.        +0.j          0.70710678+0.70710678j]]


Graph(3 vertices, 2 edges)

In [5]:
c = Circuit("R_X(0.1) 0")  # rotation around X-axis by 0.1π
print(c.to_matrix())
c.diagram("pyzx", height=120)

[[ 0.98768834-0.j          0.        -0.15643447j]
 [-0.        -0.15643447j  0.98768834-0.j        ]]


Graph(3 vertices, 2 edges)

$$
R_X(\alpha) =
\left(
\begin{array}{cc}
\cos(\alpha\pi/2) & -i \sin(\alpha\pi/2) \\
-i \sin(\alpha\pi/2) & \cos(\alpha\pi/2)
\end{array}
\right)
$$

In [6]:
c = Circuit("R_Y(0.1) 0")  # rotation around Y-axis by 0.1π
print(c.to_matrix())
c.diagram("pyzx", height=120)

[[ 0.98768834-0.j -0.15643447+0.j]
 [ 0.15643447-0.j  0.98768834-0.j]]


Graph(7 vertices, 6 edges)

$$
R_Y(\alpha) =
\left(
\begin{array}{cc}
\cos(\alpha\pi/2) & -\sin(\alpha\pi/2) \\
\sin(\alpha\pi/2) & \cos(\alpha\pi/2)
\end{array}
\right)
$$

In [7]:
c = Circuit("R_Z(0.1) 0")  # rotation around Z-axis by 0.1π
print(c.to_matrix())
c.diagram("pyzx", height=120)

[[ 0.98768834-0.15643447j  0.        +0.j        ]
 [-0.        +0.j          0.98768834+0.15643447j]]


Graph(3 vertices, 2 edges)

$$
R_Z(\alpha) =
\left(
\begin{array}{cc}
e^{-i\alpha\pi/2} & 0 \\
0 & e^{i\alpha\pi/2}
\end{array}
\right)
$$

In [8]:
c = Circuit("U3(0.1, 0.2, 0.3) 0")
print(c.to_matrix())
c.diagram("pyzx", height=120)

[[ 0.98768834-0.j         -0.09194987-0.12655814j]
 [ 0.12655814+0.09194987j  0.        +0.98768834j]]


Graph(9 vertices, 8 edges)

$$
U_3(\theta, \phi, \lambda) =
\left(
\begin{array}{cc}
\cos(\theta\pi/2) & -e^{i\lambda\pi}\sin(\theta\pi/2) \\
e^{i\phi\pi}\sin(\theta\pi/2) & e^{i(\phi+\lambda)\pi}\cos(\theta\pi/2)
\end{array}
\right)
$$

### Measurement and Reset instructions

Tsim supports all collapsing gates [supported by STIM](https://github.com/quantumlib/Stim/wiki/Stim-v1.9-Gate-Reference#collapsing-gates).

In [9]:
c = Circuit("M 0")
c.diagram("pyzx", height=120)

Graph(3 vertices, 2 edges)

Measurements (`M`, `MX`, `MY`, `MZ`) project the state into the measurement basis and write the resulting bit into the measurement record.

The measurement record can be used to conditionally apply Pauli gates:

In [10]:
c = Circuit("""
M 0
CY rec[-1] 1
""")
c.diagram("pyzx", height=170)

Graph(9 vertices, 8 edges)

The `!` operator can be used to invert the classical measurement bit that is written into the measurement record:

In [11]:
c = Circuit("M !0")
c.diagram("pyzx", height=120)

Graph(5 vertices, 4 edges)

The `MPP` instruction measures Pauli strings. The MPP can also be used in conjunction with the `!` operator to flip the classical measurement bit before writing it into the measurement record.

In [12]:
c = Circuit("MPP !Z0*Z2*Z3")
c.diagram("pyzx", height=220)

Graph(17 vertices, 16 edges)

### Noise Channels

Tsim supports all noise channels [supported by STIM](https://github.com/quantumlib/Stim/wiki/Stim-v1.9-Gate-Reference#noise-channels).

The `X_ERROR(p)` instruction is a `X` instruction that is applied with probability `p`.

In [13]:
c = Circuit("X_ERROR(0.1) 0")
c.diagram("pyzx", height=120)

Graph(3 vertices, 2 edges)

The `PAULI_CHANNEL_1(p_x, p_y, p_z)` instruction is a `X`, `Y`, and `Z` instruction that is applied with probabilities `p_x`, `p_y`, and `p_z` respectively.

In [14]:
c = Circuit("PAULI_CHANNEL_1(0.1, 0.2, 0.3) 0")
c.diagram("pyzx", height=120)

Graph(4 vertices, 3 edges)

The `PAULI_CHANNEL_2` instruction takes fifteen floats specifying the disjoint probabilities of each possible Pauli pair
that can occur (except for the non-error double identity case).
The disjoint probability arguments are (in order):

p_ix,
p_iy,
p_iz,
p_xi,
p_xx,
p_xy,
p_xz,
p_yi,
p_yx,
p_yy,
p_yz,
p_zi,
p_zx,
p_zy,
p_zz

In [15]:
c = Circuit(
    "PAULI_CHANNEL_2(0.01, 0.01, 0.03, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01) 0 1"
)
c.diagram("pyzx", height=170)

Graph(8 vertices, 6 edges)

The `DEPOLARIZE1(p)` instruction applies a randomly chosen Pauli with probability `p`.

In [16]:
c = Circuit("DEPOLARIZE1(0.01) 0")
c.diagram("pyzx", height=120)

Graph(4 vertices, 3 edges)

The `DEPOLARIZE2(p)` instruction applies a randomly chosen two-qubit Pauli with probability `p`.

In [17]:
c = Circuit("DEPOLARIZE2(0.01) 0 1")
c.diagram("pyzx", height=170)

Graph(8 vertices, 6 edges)

The `CORRELATED_ERROR(p)` instruction applies a specified Pauli product with probability `p`. If no error occurred, then a following `ELSE_CORRELATED_ERROR(p2)` instruction may apply a Pauli product with probabiliy `p2`. If no error occurs again, further `ELSE_CORRELATED_ERROR(pi)` instructions in the chain may apply a Pauli products.

In [18]:
c = Circuit("""
CORRELATED_ERROR(0.1) X0  # Apply X with probability 0.1
ELSE_CORRELATED_ERROR(0.2) Z0 Y1  # If no error occurred, apply Z0*Y1 with probability 0.2
ELSE_CORRELATED_ERROR(0.3) X1 # If still no error, apply X1 with probability 0.3
""")
c.diagram("pyzx", height=170)

Graph(9 vertices, 7 edges)

### Annotations

Tsim supports detector and observable annotations.

The `DETECTOR` instruction is only used in detector sampling mode and ignored otherwise. It instructs the detector sampler to record the XOR of classical outcomes of specified measurement bits.

In [19]:
c = Circuit("""
    M 0 0
    DETECTOR rec[-1] rec[-2]
""")
c.diagram("pyzx", height=150)

Graph(5 vertices, 5 edges)

The `OBSERVABLE_INCLUDE` instruction is only used in observable sampling mode and ignored otherwise. It instructs the detector sampler to record the XOR of the specified measurement bits.

In [20]:
c = Circuit("""
    M 0 0
    OBSERVABLE_INCLUDE(0) rec[-1] rec[-2]
""")
c.diagram("pyzx", height=150)

Graph(5 vertices, 5 edges)

## Sampling

Tsim supports multiple samplers. The first is a measurement sampler. This will simply sample bits for each measurement instruction in the circuit. Detector and observable annotations will simply be ignored by this sampler.

In [21]:
c = Circuit("""
    RX 0
    R 1
    CNOT 0 1
    M 0 1
""")
sampler = c.compile_sampler()
c.diagram("pyzx", height=170)

Graph(8 vertices, 7 edges)

In [22]:
sampler.sample(shots=5)

array([[False, False],
       [ True,  True],
       [False, False],
       [False, False],
       [False, False]])

The second sampling mode is detector sampling. This will sample detector events and observable values. Detector and observable bits can always be obtained by linear transformations of the measurement bits as return by the measurement sampler.
In practice, however, it can be much more efficient to sample detector events directly.

In [23]:
c = Circuit("""
    RX 0
    R 1
    CNOT 0 1
    M 0 1
    DETECTOR rec[-1] rec[-2]
    OBSERVABLE_INCLUDE(0) rec[-1]
""")
sampler = c.compile_detector_sampler()
c.diagram("pyzx", height=170)

Graph(10 vertices, 10 edges)

In [24]:
detectors, observables = sampler.sample(5, separate_observables=True)
print(detectors)
print(observables)

[[False]
 [False]
 [False]
 [False]
 [False]]
[[False]
 [ True]
 [False]
 [ True]
 [ True]]


Finally, Tsim allows to compute probability values for target states via the `CompiledStateProbs` sampler.

In [25]:
import numpy as np
from tsim.sampler import CompiledStateProbs

sampler = CompiledStateProbs(c)

In [26]:
sampler.probability_of(np.array([0, 0]), batch_size=1)

array([0.5], dtype=float32)

In [27]:
sampler.probability_of(np.array([0, 1]), batch_size=1)

array([0.], dtype=float32)

In [28]:
sampler.probability_of(np.array([1, 0]), batch_size=1)

array([0.], dtype=float32)

In [29]:
sampler.probability_of(np.array([1, 1]), batch_size=1)

array([0.5], dtype=float32)

## Visualization

Tsim supports multiple ways of visualizing quantum circuits.

The `timeline-svg` diagram shows the circuit as a time-ordered sequence of gates.

In [30]:
c = Circuit("""
    QUBIT_COORDS(0, 0) 0  # specifies qubit coordinates for the "timeslice-svg" diagram
    QUBIT_COORDS(1, 1) 1
    H 0
    TICK
    T 0
    H 0
    TICK
    CNOT 0 1
    TICK
    DEPOLARIZE2(0.1) 0 1
    TICK
    R_Z(0.2) 1
    TICK
    M 0 1
    DETECTOR rec[-1] rec[-2]
    OBSERVABLE_INCLUDE(0) rec[-1]
""")
c.diagram("pyzx", height=170)

Graph(16 vertices, 16 edges)

When `TICK` instructions are present, each tick can be shown as a 2D time slice with the `timeslice-svg` diagram. Here, `QUBIT_COORDS` annotations can be used to specify the 2D coordinates of the qubits.

In [31]:
c.diagram("timeslice-svg", width=800, rows=1)

With the `pyzx` argument, the circuit can be visualized using the [pyzx](https://github.com/zxcalc/pyzx) as a ZX-diagram.

In [32]:
c.diagram("pyzx");

The `pyzx-meas` and `pyzx-dets` diagrams show ZX diagrams where outputs represent probabilities of measurement outcomes for measurement and detector/observables, respectively.

In [33]:
c.diagram("pyzx-meas", scale_horizontally=2);

In [34]:
c.diagram("pyzx-dets", scale_horizontally=1.5);

## Detector Error Models

Tsim allows to compute detector error models from a circuit. The method `Circuit.detector_error_model()` computes a `stim.DetectorErrorModel` from the circuit. As opposed to Stim, detectors and observables need not be deterministic.

In [35]:
c = Circuit("""
    RX 0
    R 1
    CNOT 0 1
    DEPOLARIZE1(0.1) 0 1
    M 0 1
    DETECTOR rec[-1] rec[-2]
    DETECTOR rec[-1]
""")
c.detector_error_model()

stim.DetectorErrorModel('''
    error(0.0666667) D0
    error(0.0666667) D0 D1
    error(0.5) D1
''')